In [18]:
import pickle
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

In [19]:
with open('train_sentences.pkl', 'rb') as f:
    train_sentences = pickle.load(f)

In [20]:
maxx = 0
for s in train_sentences:
    #print(s)
    c = 0
    for w in s:
        #print(w)
        if w[2]: c += 1
    if c > maxx: maxx = c

print(maxx)

36


In [21]:
word_set = set()

for s in train_sentences:
    for t in s:
        word_set.add(t[0])
# print(word_set)

words = list(word_set)
words.append("ENDPAD")

n_words = len(words); n_words
max_len = 35
word2idx = {w: i+1 for i, w in enumerate(words)}

In [22]:
X = [[word2idx[w[0]] for w in s] for s in train_sentences]
# print (X)
X = pad_sequences(maxlen=max_len, sequences=X, padding="post", value=0)
y = [[w[2] for w in s] for s in train_sentences]
y = pad_sequences(maxlen=max_len, sequences=y, padding="post", value=0)

In [23]:
X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.1, shuffle=False)

In [24]:
from keras import backend as K

def recall(y_true, y_pred):
    """Recall metric.

    Only computes a batch-wise average of recall.

    Computes the recall, a metric for multi-label classification of
    how many relevant items are selected.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall


def precision(y_true, y_pred):
    """Precision metric.

    Only computes a batch-wise average of precision.

    Computes the precision, a metric for multi-label classification of
    how many selected items are relevant.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1(y_true, y_pred):
    prec = precision(y_true, y_pred)
    rec = recall(y_true, y_pred)
    return 2*((prec*rec)/(prec+rec+K.epsilon()))

In [25]:
from keras.models import Model, Input, Sequential
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional, Flatten
from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
#from keras_contrib.layers import CRF

earlyStopper = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=1, mode='auto')


model = Sequential()
model.add(Embedding(input_dim=n_words, output_dim=max_len, input_length=max_len))
model.add(Dropout(0.1))
# model.add(Bidirectional(LSTM(units=max_len, return_sequences=True, recurrent_dropout=0.1)))
# model.add(Bidirectional(LSTM(units=max_len, return_sequences=True, recurrent_dropout=0.1)))
model.add(LSTM(units=max_len, return_sequences=True, recurrent_dropout=0.1))
# model.add(LSTM(units=max_len, return_sequences=True, recurrent_dropout=0.1))
model.add(Flatten())
model.add(Dense(activation='sigmoid', output_dim=max_len))


# # model = Model(inputt, out)
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy", f1, precision, recall])

history = model.fit(X_tr, y_tr, batch_size=32, epochs=50, validation_split=0.1, verbose=1, callbacks=[])

hist = pd.DataFrame(history.history)
plt.figure(figsize=(12,12))
# plt.plot(hist["acc"] , label='acc')
# plt.plot(hist["val_acc"] , label='val_acc')

# plt.plot(hist['loss'], label='loss')
# plt.plot(hist['val_loss'], label='val_loss')
plt.plot(hist['precision'], label='precision')
plt.plot(hist['val_precision'], label='val_precision')
plt.plot(hist['recall'], label='recall')
plt.plot(hist['val_recall'], label='val_recall')
plt.legend()
plt.show()

C:\Users\Sandaru\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="sigmoid", units=35)`


Train on 324 samples, validate on 37 samples
Epoch 1/50
324/324 [==============================] - 2s 7ms/step - loss: 0.6417 - acc: 0.8618 - f1: 0.0433 - precision: 0.0359 - recall: 0.1258 - val_loss: 0.5225 - val_acc: 0.9506 - val_f1: 0.0000e+00 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/50
324/324 [==============================] - 0s 1ms/step - loss: 0.3176 - acc: 0.9584 - f1: 0.0000e+00 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 0.1873 - val_acc: 0.9506 - val_f1: 0.0000e+00 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 3/50
324/324 [==============================] - 0s 1ms/step - loss: 0.1881 - acc: 0.9584 - f1: 0.0000e+00 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 0.1759 - val_acc: 0.9506 - val_f1: 0.0000e+00 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 4/50
324/324 [==============================] - 0s 1ms/step - loss: 0.1453 - acc: 0.9584 - f1: 0.0000e+00 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_

324/324 [==============================] - 0s 1ms/step - loss: 0.0611 - acc: 0.9764 - f1: 0.6372 - precision: 0.8373 - recall: 0.5262 - val_loss: 0.2446 - val_acc: 0.9421 - val_f1: 0.0246 - val_precision: 0.0270 - val_recall: 0.0225
Epoch 33/50
324/324 [==============================] - 0s 1ms/step - loss: 0.0591 - acc: 0.9757 - f1: 0.6546 - precision: 0.8045 - recall: 0.5616 - val_loss: 0.2393 - val_acc: 0.9459 - val_f1: 0.0520 - val_precision: 0.2000 - val_recall: 0.0374
Epoch 34/50
324/324 [==============================] - 0s 1ms/step - loss: 0.0551 - acc: 0.9789 - f1: 0.7040 - precision: 0.8551 - recall: 0.6069 - val_loss: 0.2428 - val_acc: 0.9452 - val_f1: 0.0516 - val_precision: 0.1712 - val_recall: 0.0374
Epoch 35/50
324/324 [==============================] - 1s 2ms/step - loss: 0.0534 - acc: 0.9804 - f1: 0.7459 - precision: 0.8105 - recall: 0.6958 - val_loss: 0.2464 - val_acc: 0.9444 - val_f1: 0.0512 - val_precision: 0.1506 - val_recall: 0.0374
Epoch 36/50
324/324 [===========

NameError: name 'pd' is not defined

In [26]:
from sklearn.metrics import recall_score, precision_score, f1_score
import numpy as np

threshold = 0.5

pred_y_te = model.predict(X_te)
# pred_y_te = np.round(pred_y_te)
pred_y_te = np.array([ [0 if e < threshold else 1 for e in p] for p in pred_y_te ])

pred_y_te[6]

print('Precision : ' + str(precision_score(y_te, pred_y_te, average='micro')))
print('Recall : ' + str(recall_score(y_te, pred_y_te, average='micro')))
print('F1-score : ' + str(f1_score(y_te, pred_y_te, average='micro')))

Precision : 0.47619047619
Recall : 0.294117647059
F1-score : 0.363636363636
